In [2]:
import numpy as np

In [3]:
def padAmount(image_size, filter_size, stride = 1, mode = 'valid'):
    if(mode == 'same'):
        lower_bound = (image_size-1)*stride - (image_size-filter_size)
        return int((lower_bound + lower_bound%2)/2)
    elif(mode == 'valid'):
        return 0
    else:
        print('invalid input')
        return -1
    

In [4]:
from PIL import Image

In [34]:
img1 = Image.open('5.jpg').convert('L')
im1_array = np.array(img1)
#info = np.iinfo(im1_array.dtype)
#im1_array = im1_array.astype(np.integer)/info.max

kernel = np.array(
    [[0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]]
)
#kernel = 4/6*kernel

stride = int(input("please enter the value of stride: "))
mode = input("please enter the mode of the size of the output image: ")

paddedim1_array = pad_image(im1_array, kernel, stride)
out_img1 = Image.fromarray(convolve_matrix(paddedim1_array, kernel, stride))
out_img1 = out_img1.convert('L')
out_img1.show()

please enter the value of stride: 1
please enter the mode of the size of the output image: awsfasfa


In [8]:
# padding section:
def pad_image(im1_array, kernel, stride):
    imgsize_x = im1_array.shape[0]
    imgsize_y = im1_array.shape[1]
    kernel_size = kernel.shape[0]
    pad_x = padAmount(imgsize_x, kernel_size, stride, mode='same')
    pad_y = padAmount(imgsize_y, kernel_size, stride, mode='same')
    paddedim1_array = np.pad(im1_array, (pad_x, pad_y))
    padded_img1 = Image.fromarray(paddedim1_array)
    
    return paddedim1_array

In [9]:
 def convolve_matrix(img_arr, kernel, stride = 1):
    kernel_size = kernel.shape[0]
    slice_x = np.arange(kernel_size)
    slice_y = np.arange(kernel_size)
    for_leni = img_arr.shape[0]-kernel_size+1
    for_lenj = img_arr.shape[1]-kernel_size+1
    output_img = []
    for i in range(0, for_leni, stride):
        row = []
        for j in range(0, for_lenj, stride):
            a = slice_x+i
            b = slice_y+j
            row.append(np.sum(img_arr[np.ix_(a,b)]*kernel))
        output_img.append(row)
    return np.array(output_img)

In [13]:
img1.show()

In [11]:
#edge detection:
[[-1, -1, -1],
[-1,  8, -1],
[-1, -1, -1]]
#sharpening:
[[0, -1, 0],
[-1, 5, -1],
[0, -1, 0]]
#normalize box blur:
1/9*[[1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]]
#gaussian blur 5*5:
1/256 *  [[1, 4, 6, 4, 1],
         [4, 16, 24, 16, 4],
         [6, 24, 36, 24, 6],
         [4, 16, 24, 16, 4],
         [1, 4, 6, 4, 1]]
#unsharp masking 5*5:
-1/256 * [[1, 4, 6, 4, 1],
         [4, 16, 24, 16, 4],
         [6, 24, -476, 24, 6],
         [4, 16, 24, 16, 4],
         [1, 4, 6, 4, 1]]
#embossing filter
[[-1, -1, 0],
 [-1, 0 , 1],
 [0, 1, 1]]
#embossing filter
 1/9*[[1, 1, 1],
     [1, 1 , 1],
     [1, 1, 1]]
#intensified sharpening
[[-1, -1, -1],
 [-1, 9, -1],
 [-1, -1, -1]]

IndentationError: unexpected indent (<ipython-input-11-17cede5d1f40>, line 30)

In [33]:
out_img1.save("5_out.png")